In [3]:
import numpy as np
import pandas as pd
from random import sample
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from plotly import graph_objects as go
from collections import defaultdict
import datetime

In [ ]:
data = pd.read_csv('program.csv', sep=';')
scores = defaultdict(int)
data.head()

In [ ]:


def get_layout(title='', height=500):
    standart_layout = go.Layout({
        'height': height,
        'width': 600,
        'title': title,
        'template': 'plotly_white'
    })
    
    return standart_layout


def get_random_ticket():
    return sample([i for i in range(data.shape[0])], k=1)[0]

CUR_TICKET = 0
STATISTIC = defaultdict(int)
G_CLICKS = 0
B_CLICKS = 0
S_CLICKS = 0
CUR_DATE = str(datetime.datetime.today()).split()[0]
for i in range(data.shape[0]):
    STATISTIC[i] = [0, 0]
dates = [str(datetime.datetime.today() - datetime.timedelta(days=1)).split()[0],
         str(datetime.datetime.today()).split()[0]]

# Front

In [2]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    html.H1('Start'),
    
    html.Div([dcc.Graph(figure=go.Figure(data=[], layout=get_layout('Распределение по билетам')),
                       id='main_hist',
                       style={'display': 'inline-block'}),
              dcc.Graph(figure=go.Figure(data=[], layout=get_layout('Прогресс')),
                       id='main_prog',
                       style={'display': 'inline-block'})
             ],
             style={'display': 'inline-block'}
    ),
    html.Div([html.Button('Update', id='update_button')]),
    html.Hr(),
    html.H3(id='display_ticket', 
             children="Нажми start",
             style={'text-align': 'center'}
            ),
    html.Div(
        [html.Button('Start', id='start_button', value='Start')],
        style={'content-align': 'center'}
    ),
    html.Div([dcc.Graph(figure=go.Figure(data=[], layout=get_layout('Статистика по билету')), id='ticket_stat')],
             style={'display': 'inline-block', 'centering': 'middle'},
             ),
    html.Hr(),
    html.Div([
        html.Button('Ok', id='good_button'),
        html.Span(),
        html.Button('Не Ok', id='bad_button')
    ], 
    style={'display': 'inline-block', 'centering': 'middle'}
    )
    
])


NameError: name 'dash' is not defined

# Back

In [ ]:
@app.callback(
    [Output('display_ticket', 'children'),
     Output('ticket_stat', 'figure')],
    [Input('start_button', 'n_clicks'),
     Input('good_button', 'n_clicks'), 
     Input('bad_button', 'n_clicks')]
)
def get_statistic(n_clicks, g_n_clicks=None, b_n_clicks=None):
    
    global G_CLICKS, B_CLICKS, STATISTIC, CUR_TICKET, S_CLICKS
    if n_clicks is not None:
        if n_clicks != S_CLICKS:
            S_CLICKS += 1
            CUR_TICKET = get_random_ticket()
    if g_n_clicks is not None:
        if b_n_clicks is not None:
            if G_CLICKS != g_n_clicks:
                to_update = STATISTIC[CUR_TICKET]
                to_update[0] += 1
                STATISTIC[CUR_TICKET] = to_update
                G_CLICKS += 1
                print(to_update)
                
            if B_CLICKS != b_n_clicks:
                to_update = STATISTIC[CUR_TICKET]
                to_update[1] += 1
                print(to_update)
                STATISTIC[CUR_TICKET] = to_update
                B_CLICKS += 1
                
        if G_CLICKS != g_n_clicks:
            to_update = STATISTIC[CUR_TICKET]
            to_update[0] += 1
            print(to_update)
            STATISTIC[CUR_TICKET] = to_update
            G_CLICKS += 1
            
    
    if b_n_clicks is not None:   
        if B_CLICKS != b_n_clicks:
            to_update = STATISTIC[CUR_TICKET]
            to_update[1] += 1
            print(to_update)
            STATISTIC[CUR_TICKET] = to_update
            B_CLICKS += 1
    
    x = ['Правильно', 'Неправильно']           
    y = STATISTIC[CUR_TICKET]
    bar = go.Bar({
        'x': x,
        'y': y,
        'marker': {
            'color': ['green', 'red']
        }
    })
    return data['text'][CUR_TICKET], go.Figure(data=[bar], layout=get_layout('Статистика по билету'))
    
@app.callback(
    [Output('main_hist', 'figure'),
    Output('main_prog', 'figure')],
    [Input('update_button', 'n_clicks')]
)
def update_stat(n_clicks):
    if n_clicks is not None:
        x = data.number.values
        y_p = [v[0] for k, v in STATISTIC.items()]
        y_n = [v[1] for k, v in STATISTIC.items()]
        trace_positive = go.Bar({
            'x': x,
            'y': y_p,
            'marker': {
                'color': 'green'
            },
            'name': 'Правильно'
            
        })
        trace_negative = go.Bar({
            'x': x,
            'y': y_n,
            'marker': {
                'color': 'red'
            },
            'name': 'Неправильно'
        })
        layout = go.Layout({
            'height': 500,
            'width': 600,
            'title': 'Распределение по билетам',
            'template': 'plotly_white',
            'barmode': 'stack'
        })
        fig_h = go.Figure(data= [trace_positive, trace_negative], layout=layout)
        global dates, CUR_DATE
        if CUR_DATE != str(datetime.datetime.today()).split()[0]:
            dates.append(str(datetime.datetime.today()).split()[0])
            CUR_DATE = str(datetime.datetime.today()).split()[0]
        x_dates = dates
        # TODO
        
  
        
        return [fig_h, go.Figure(data=[], layout=get_layout('Прогресс'))]

In [ ]:
CUR_TICKET = 0
G_CLICKS = 0
B_CLICKS = 0
S_CLICKS = 0
app.server.run(host='localhost', port=8899)